# Инициализация

Загружаем библиотеки необходимые для выполнения кода ноутбука.

In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
plt.rcParams['figure.figsize'] = (10,5)

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [5]:
tracks = pd.read_parquet("./datasets/tracks.parquet")

In [46]:
catalog_names = pd.read_parquet("./datasets/catalog_names.parquet")

In [50]:
interactions = pd.read_parquet("./datasets/interactions.parquet")

# Обзор данных

Проверяем данные, есть ли с ними явные проблемы.

In [6]:
# функция для знакомства с данными
def df_info(df):
    display("Фрагмент датасета")
    display(df.sample(5))
    display("-------------------------")
    display("Проверяем на наличие пропущенных значение")
    display(df.isna().sum())
    display("-------------------------")
    display("Информация о датасете")
    display(df.info())
    display("-------------------------")
    display("Размерность")
    display(df.shape)

In [7]:
df_info(tracks)

'Фрагмент датасета'

,track_id,albums,artists,genres
998600,100031842,[20945874],[9320307],"[44, 75]"
75025,670260,"[72748, 75642]",[1546],"[28, 163]"
965432,89305127,[17503506],[7720909],[16]
772164,59485362,[9117459],[8066072],[68]
470484,32690339,[3986617],[94267],[11]


'-------------------------'

'Проверяем на наличие пропущенных значение'

track_id    0
albums      0
artists     0
genres      0
dtype: int64

...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   track_id  1000000 non-null  int64 
 1   albums    1000000 non-null  object
 2   artists   1000000 non-null  object
 3   genres    1000000 non-null  object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB


None

'-------------------------'

'Размерность'

(1000000, 4)

С виду явных проблем нет, идентификаторы менять не стоит. Но видно что данные в столбцах кроме track_id содержат массивы, очевидно что пустой массив это не пропущенное значение, хотя данных не содержит. Проведу проверку.

In [13]:
# функция для поиска пустых массивов.
def missed_value(df, column):
    df_missed_value =df[df[column].apply(lambda x: len(x) == 0)]
    return df_missed_value

In [14]:
missed_value(tracks, 'artists')

,track_id,albums,artists,genres
146031,3599314,[389925],[],"[68, 86]"
146163,3599591,[389944],[],"[68, 86]"
177544,4790215,[533785],[],"[68, 86]"
218922,10063296,[1083742],[],"[11, 20]"
231455,12122918,[1315834],[],[68]
...,...,...,...,...
999868,101201589,[12649219],[],"[62, 77]"
999876,101219571,[9444321],[],"[62, 83, 77]"
999901,101243616,[11700332],[],"[62, 93]"
999920,101292599,[9331881],[],"[62, 90, 77, 84]"


In [15]:
missed_value(tracks, 'albums')

,track_id,albums,artists,genres
310821,20200372,[],[],[]
310826,20200380,[],[],[]
312469,20305116,[],[],[]
312474,20305121,[],[],[]
320353,20756854,[],[],[]
326588,21196099,[],[],[]
326592,21196103,[],[],[]
326594,21196105,[],[],[]
326596,21196107,[],[],[]
326598,21196109,[],[],[]


In [16]:
missed_value(tracks, 'genres')

,track_id,albums,artists,genres
490,2520,"[249, 163801, 163802, 324616, 17542430, 189498...",[2259],[]
3069,16776,"[1479, 5319, 6093, 8467, 10641, 90867, 163801,...",[978],[]
3077,16801,"[1479, 163801, 9287549, 9377183, 9424994, 1201...",[10520],[]
4192,23752,"[2001, 3722, 4973, 6104, 6558, 23866, 37052, 7...",[13373],[]
6645,38012,"[3147, 60099, 530836, 1115289, 2490724, 818446...",[16758],[]
...,...,...,...,...
997667,99412992,[20764551],[10029101],[]
999192,100497082,[21098340],[15936615],[]
999703,100923319,[21233460],[5271861],[]
999704,100923322,[21233460],[5271861],[]


В датафрейме есть строки с неизвестными исполнителями, альбомами и жанрами. Для принятия решения что делать с этими строками изучу какой процент от общих они составляют.

In [30]:
#tracks = tracks.explode('albums')
#tracks = tracks.explode('artists')
#tracks = tracks.explode('genres')

In [33]:
missing_values = tracks[
    (tracks['artists'].str.len() == 0) | (tracks['albums'].str.len() == 0) | (tracks['genres'].str.len() == 0)]

In [39]:
percent_missing_values = len(missing_values) / len(tracks) * 100
print(f"Процент данных в missing_values от общего количества данных в tracks: {percent_missing_values:.1f}%")

Процент данных в missing_values от общего количества данных в tracks: 1.9%


Менее 3%, считаю что могу их просто удалить.

In [40]:
indexes_to_drop = missing_values.index

In [41]:
tracks = tracks.drop(indexes_to_drop).reset_index(drop=True)

In [43]:
missed_value(tracks, 'albums')

,track_id,albums,artists,genres


In [42]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980977 entries, 0 to 980976
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   track_id  980977 non-null  int64 
 1   albums    980977 non-null  object
 2   artists   980977 non-null  object
 3   genres    980977 non-null  object
dtypes: int64(1), object(3)
memory usage: 29.9+ MB


In [44]:
# уборочка
del tracks

In [47]:
df_info(catalog_names)

'Фрагмент датасета'

,id,type,name
1103491,18539272,track,Ваза и пион
36530,100980,album,Naked & Warm
1722965,77373052,track,Неоновый потоп
274180,4401475,album,Sweet Bossa Nova
761145,5125722,artist,Костя Битеев


'-------------------------'

'Проверяем на наличие пропущенных значение'

id      0
type    0
name    0
dtype: int64

...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812471 entries, 0 to 1812470
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   type    object
 2   name    object
dtypes: int64(1), object(2)
memory usage: 41.5+ MB


None

'-------------------------'

'Размерность'

(1812471, 3)

In [49]:
del catalog_names

In [51]:
df_info(interactions)

'Фрагмент датасета'

,user_id,track_id,track_seq,started_at
119,953300,669091,120,2022-10-19
296,1238971,38634578,297,2022-10-06
230,1180163,2810298,231,2022-08-19
24,602369,178531,25,2022-11-02
73,33984,96366199,74,2022-12-29


'-------------------------'

'Проверяем на наличие пропущенных значение'

user_id       0
track_id      0
track_seq     0
started_at    0
dtype: int64

...

<class 'pandas.core.frame.DataFrame'>
Index: 222629898 entries, 0 to 291
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   user_id     int32         
 1   track_id    int32         
 2   track_seq   int16         
 3   started_at  datetime64[ns]
dtypes: datetime64[ns](1), int16(1), int32(2)
memory usage: 5.4 GB


None

'-------------------------'

'Размерность'

(222629898, 4)

# Выводы

Приведём выводы по первому знакомству с данными:
- есть ли с данными явные проблемы,
- какие корректирующие действия (в целом) были предприняты.

# === ЭТАП 2 ===

# EDA

Распределение количества прослушанных треков.

In [26]:
interactions.sample(3).T

,704,280,1241
user_id,727798,206585,315912
track_id,56016978,36827140,71733873
track_seq,705,281,1242
started_at,2022-10-17 00:00:00,2022-09-01 00:00:00,2022-11-29 00:00:00


In [30]:
len(interactions[(interactions.duplicated(['user_id','track_id']) == True)])

0

In [ ]:
track_play_counts = df.groupby('track_id').size().reset_index(name='play_count')
print(track_play_counts)

In [27]:
# оценка взаимодействия пользователей с трекаками
print(interactions[["user_id", "track_id", "track_seq", "started_at"]].sample(5).sort_values("user_id").set_index(["user_id", "track_id"]))

                  track_seq started_at
user_id track_id                      
369734  79184551       1115 2022-11-21
626108  19622237        290 2022-07-30
728843  795836          129 2022-07-04
1089924 77296031        167 2022-09-29
1279900 33355173         39 2022-06-10


In [32]:
interactions.describe(include='all').T

,count,mean,min,25%,50%,75%,max,std
user_id,222629898.0,687576.660624,0.0,343371.0,687973.0,1031127.0,1374582.0,396903.293288
track_id,222629898.0,36536224.961649,26.0,14808489.0,35524737.0,56511373.0,101521819.0,26617816.837147
track_seq,222629898.0,462.140293,1.0,56.0,181.0,506.0,16637.0,825.731213
started_at,222629898,2022-08-29 16:39:44.541336320,2022-01-01 00:00:00,2022-07-02 00:00:00,2022-09-15 00:00:00,2022-11-09 00:00:00,2022-12-31 00:00:00,NaN


In [28]:
# Подсчет количества прослушанных треков для каждого user_id
track_counts = interactions.groupby('user_id')['track_id'].count()

# Построение гистограммы распределения количества прослушанных треков
track_counts.head()

user_id
0     26
1     36
2     14
3     33
4    256
Name: track_id, dtype: int64

Наиболее популярные треки

Наиболее популярные жанры

Треки, которые никто не прослушал

# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

# === ЭТАП 3 ===

# Загрузка данных

Если необходимо, то загружаем items.parquet, events.parquet.

# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

# Топ популярных

Рассчитаем рекомендации как топ популярных.

# Персональные

Рассчитаем персональные рекомендации.

# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

# Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

# Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.